<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/qdrant_arize.png" width="500"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tuning a RAG Pipeline using Qdrant and Arize Phoenix</h1>

ℹ️ This notebook requires an OpenAI API key.

### **1. Import Relevant Packages**

In [1]:
import os

# Setup projects
SIMPLE_RAG_PROJECT = "simple-rag"
HYBRID_RAG_PROJECT = "hybrid-rag"
os.environ["PHOENIX_PROJECT_NAME"] = SIMPLE_RAG_PROJECT

In [2]:
import datetime
import json
import os
import pickle
import ssl
import time
import urllib
from getpass import getpass
from urllib.request import urlopen

import certifi
import nest_asyncio
import openai
import pandas as pd
import phoenix as px
import requests
from bs4 import BeautifulSoup
from llama_index.core import (
    ServiceContext, StorageContext, download_loader,
    load_index_from_storage, set_global_handler
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.graph_stores.simple import SimpleGraphStore
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from phoenix.evals import (
    HallucinationEvaluator, OpenAIModel, QAEvaluator,
    RelevanceEvaluator, run_evals
)
from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations
from tqdm import tqdm

import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import PointStruct

nest_asyncio.apply()  # needed for concurrent evals in notebook environments
pd.set_option("display.max_colwidth", 1000)

### **2. Launch Phoenix**
You can run Phoenix in the background to collect trace data emitted by any LlamaIndex application that has been instrumented with the OpenInferenceTraceCallbackHandler. Phoenix supports LlamaIndex's one-click observability which will automatically instrument your LlamaIndex application! You can consult our integration guide for a more detailed explanation of how to instrument your LlamaIndex application.

Launch Phoenix and follow the instructions in the cell output to open the Phoenix UI (the UI should be empty because we have yet to run the LlamaIndex application).

In [3]:
session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


Be sure to enable phoenix as your global handler for tracing!

In [4]:
set_global_handler("arize_phoenix")

### **3. Setup your openai key and retrieve the documents to be used**

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

### **4. Retrieve the documents / dataset to be used**

In [7]:
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset("atitaarora/qdrant_doc", split="train")

In [8]:
dataset.info

DatasetInfo(description='', citation='', homepage='', license='', features={'text': Value(dtype='string', id=None), 'source': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='csv', dataset_name='qdrant_doc', config_name='default', version=0.0.0, splits={'train': SplitInfo(name='train', num_bytes=1767967, num_examples=240, shard_lengths=None, dataset_name='qdrant_doc')}, download_checksums={'hf://datasets/atitaarora/qdrant_doc@8d859890840f65337c38e96d660b81b1441bbecd/documents.csv': {'num_bytes': 1777260, 'checksum': None}}, download_size=1777260, post_processing_size=None, dataset_size=1767967, size_in_bytes=3545227)

### **5. Definition of global chunk properties and chunk processing**
Processing each document with desired **TEXT_SPLITTER_ALGO , CHUNK_SIZE , CHUNK_OVERLAP** etc

In [11]:
## Global config for chunk processing
CHUNK_SIZE = 512 #1000
CHUNK_OVERLAP = 50

### **6. Process dataset as langchain (or llamaindex) document for further processing**

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument
from llama_index.core import Document

## Split and process the document chunks from the given dataset

def process_document_chunks(dataset,chunk_size,chunk_overlap):
    langchain_docs = [
        LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
        for doc in tqdm(dataset)
    ]

    # could showcase another variation of processed documents
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        add_start_index=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    docs_processed = []
    for doc in langchain_docs:
        docs_processed += text_splitter.split_documents([doc])

    ## Converting Langchain document chunks above into Llamaindex Document for ingestion
    llama_documents = [
        Document.from_langchain_format(doc)
        for doc in docs_processed
    ]
    return llama_documents

In [13]:
documents = process_document_chunks(dataset, CHUNK_SIZE, CHUNK_OVERLAP)
len(documents)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 240/240 [00:00<00:00, 8416.59it/s]


4431

### **7. Setting up Qdrant and Collection**

We first set up the qdrant client and then create a collection so that our data may be stored.

In [14]:
##Uncomment to initialise qdrant client in memory
#client = qdrant_client.QdrantClient(
#    location=":memory:",
#)

##Uncomment below to connect to Qdrant Cloud
client = QdrantClient(
    os.environ.get("QDRANT_URL"), 
    api_key=os.environ.get("QDRANT_API_KEY"),
)

## Uncomment below to connect to local Qdrant
#client = qdrant_client.QdrantClient("http://localhost:6333")

In [20]:
## Collection Name 
COLLECTION_NAME = "qdrant_docs_arize_dense"

In [21]:
## General Collection level operations

## Get information about existing collections 
client.get_collections()

## Get information about specific collection
#collection_info = client.get_collection(COLLECTION_NAME)
#print(collection_info)

## Deleting collection, if need be
#client.delete_collection(COLLECTION_NAME)

CollectionsResponse(collections=[CollectionDescription(name='qdrant_docs_arize_hybrid')])

In [12]:
## Declaring the intended Embedding Model with Fastembed
from fastembed.embedding import TextEmbedding

pd.DataFrame(TextEmbedding.list_supported_models())

2024-04-16 17:55:12.207 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


,model,dim,description,size_in_GB,sources
0,BAAI/bge-base-en,768,Base English model,0.420,{'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz'}
1,BAAI/bge-base-en-v1.5,768,"Base English model, v1.5",0.210,"{'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz', 'hf': 'qdrant/bge-base-en-v1.5-onnx-q'}"
2,BAAI/bge-large-en-v1.5,1024,"Large English model, v1.5",1.200,{'hf': 'qdrant/bge-large-en-v1.5-onnx'}
3,BAAI/bge-small-en,384,Fast English model,0.130,{'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz'}
4,BAAI/bge-small-en-v1.5,384,Fast and Default English model,0.067,{'hf': 'qdrant/bge-small-en-v1.5-onnx-q'}
5,BAAI/bge-small-zh-v1.5,512,Fast and recommended Chinese model,0.090,{'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz'}
6,sentence-transformers/all-MiniLM-L6-v2,384,"Sentence Transformer model, MiniLM-L6-v2",0.090,"{'url': 'https://storage.googleapis.com/qdrant-fastembed/sentence-transformers-all-MiniLM-L6-v2.tar.gz', 'hf': 'qdrant/all-MiniLM-L6-v2-onnx'}"
7,sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2,384,"Sentence Transformer model, paraphrase-multilingual-MiniLM-L12-v2",0.220,{'hf': 'qdrant/paraphrase-multilingual-MiniLM-L12-v2-onnx-Q'}
8,nomic-ai/nomic-embed-text-v1,768,8192 context length english model,0.520,{'hf': 'nomic-ai/nomic-embed-text-v1'}
9,nomic-ai/nomic-embed-text-v1.5,768,8192 context length english model,0.520,{'hf': 'nomic-ai/nomic-embed-text-v1.5'}


### **8. Document Embedding processing and Ingestion**

This example uses a `QdrantVectorStore` and creates a new collection to work fully connected with Qdrant but you can use whatever LlamaIndex application you like.

In [22]:
## Initializing the space to work with llama-index and related settings
import llama_index
from llama_index.core import Settings
from llama_index.vector_stores.qdrant import QdrantVectorStore
from phoenix.trace import suppress_tracing
## Uncomment it if you'd like to use FastEmbed instead of OpenAI
## For the complete list of supported models,
##please check https://qdrant.github.io/fastembed/examples/Supported_Models/
from llama_index.embeddings.fastembed import FastEmbedEmbedding

vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

##Uncomment if using FastEmbed
Settings.embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

## Uncomment it if you'd like to use OpenAI Embeddings instead of FastEmbed
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

Settings.llm = OpenAI(model="gpt-4-1106-preview", temperature=0.0)


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [23]:
## Note: Indexing block is to be used when you're creating a new collection 
## If using an existing collection, Skip this block and execute the next one instead.

from phoenix.trace import suppress_tracing
with suppress_tracing():
  dense_vector_index = VectorStoreIndex.from_documents(
      documents,
      storage_context=storage_context,
      show_progress=True
  )

Parsing nodes:   0%|          | 0/4431 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/335 [00:00<?, ?it/s]

### **8a. Connecting to existing Collection**

This example uses a `QdrantVectorStore` and uses the previously generated collection to work fully connected with Qdrant.

In [24]:
## Note: Execute this block when using an existing collection
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from llama_index.core.indices.vector_store import VectorIndexRetriever
dense_vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)
dense_vector_index = VectorStoreIndex.from_vector_store(vector_store=dense_vector_store)

In [25]:
## Sanity Check : Count the total number of documents processed equals number of documents in the collection
client.count(collection_name=COLLECTION_NAME)

CountResult(count=4431)

### **9.Running an example query and printing out the response.**

In [26]:
##Initialise retriever to interact with the Qdrant collection
dense_retriever = VectorIndexRetriever(
    index=dense_vector_index,
    vector_store_query_mode=VectorStoreQueryMode.DEFAULT,
    similarity_top_k=2
)

In [27]:
## let's try out a sample query using our dense retriever
response = dense_retriever.retrieve("What is a Merge Optimizer?")
for i, node in enumerate(response):
    print(i + 1, node.text, end="\n\n")

1 Such segments, for example, are created as copy-on-write segments during optimization itself.



It is also essential to have at least one small segment that Qdrant will use to store frequently updated data.

On the other hand, too many small segments lead to suboptimal search performance.



There is the Merge Optimizer, which combines the smallest segments into one large segment. It is used if too many segments are created.

2 ---

title: Optimizer

weight: 70

aliases:

  - ../optimizer

---



# Optimizer



It is much more efficient to apply changes in batches than perform each change individually, as many other databases do. Qdrant here is no exception. Since Qdrant operates with data structures that are not always easy to change, it is sometimes necessary to rebuild those structures completely.



Storage optimization in Qdrant occurs at the segment level (see [storage](../storage)).



In [28]:
# We can view the above data in the UI
px.active_session().view()

📺 Opening a view to the Phoenix app. The app is running at http://localhost:6006/


### **10. Run Your Query Engine and View Your Traces in Phoenix**

We've compiled a list of the baseline questions about Qdrant. Let's download the sample queries and take a look.

In [29]:
## Loading the Eval dataset
from datasets import load_dataset
qdrant_qa = load_dataset("atitaarora/qdrant_doc_qna", split="train")
qdrant_qa_question = qdrant_qa.select_columns(['question'])

In [30]:
qdrant_qa_question['question'][:10]

['What is vaccum optimizer ?',
 'Tell me about ‘always_ram’ parameter?',
 'What is difference between scalar and product quantization?',
 'What is ‘best_score’ strategy?',
 'How does oversampling helps?',
 'What is the purpose of ‘CreatePayloadIndexAsync’?',
 'What is the purpose of ef_construct in HNSW ?',
 'How do you use ‘ordering’ parameter?',
 'What is significance of ‘on_disk_payload’ setting?',
 'What is the impact of ‘write_consistency_factor’ ?']

In [31]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

#define response synthesizer
response_synthesizer = get_response_synthesizer()

#assemble query engine for dense retriever
dense_query_engine = RetrieverQueryEngine(
                     retriever=dense_retriever,
                     response_synthesizer=response_synthesizer,)
#query_engine = index.as_query_engine()
for query in tqdm(qdrant_qa_question['question'][:10]):
    try:
      dense_query_engine.query(query)
    except Exception as e:
      pass

  0%|                                                                                                                                        | 0/10 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:15<00:00,  7.59s/it]


Check the Phoenix UI as your queries run. Your traces should appear in real time.

Open the Phoenix UI with the link below if you haven't already and click through the queries to better understand how the query engine is performing. For each trace you will see a break

Phoenix can be used to understand and troubleshoot your by surfacing:
 - **Application latency** - highlighting slow invocations of LLMs, Retrievers, etc.
 - **Token Usage** - Displays the breakdown of token usage with LLMs to surface up your most expensive LLM calls
 - **Runtime Exceptions** - Critical runtime exceptions such as rate-limiting are captured as exception events.
 - **Retrieved Documents** - view all the documents retrieved during a retriever call and the score and order in which they were returned
 - **Embeddings** - view the embedding text used for retrieval and the underlying embedding model
LLM Parameters - view the parameters used when calling out to an LLM to debug things like temperature and the system prompts
 - **Prompt Templates** - Figure out what prompt template is used during the prompting step and what variables were used.
 - **Tool Descriptions** - view the description and function signature of the tools your LLM has been given access to
 - **LLM Function Calls** - if using OpenAI or other a model with function calls, you can view the function selection and function messages in the input messages to the LLM.

<img src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/RAG_trace_details.png" alt="Trace Details View on Phoenix" style="width:100%; height:auto;">

In [20]:
print(f"🚀 Open the Phoenix UI if you haven't already: {session.url}")

🚀 Open the Phoenix UI if you haven't already: http://localhost:6006/


### **11. Export and Evaluate Your Trace Data**
You can export your trace data as a pandas dataframe for further analysis and evaluation.

In this case, we will export our retriever spans into two separate dataframes:

queries_df, in which the retrieved documents for each query are concatenated into a single column, retrieved_documents_df, in which each retrieved document is "exploded" into its own row to enable the evaluation of each query-document pair in isolation. This will enable us to compute multiple kinds of evaluations, including:

relevance: Are the retrieved documents grounded in the response? Q&A correctness: Are your application's responses grounded in the retrieved context? hallucinations: Is your application making up false information?

In [32]:
queries_df = get_qa_with_reference(px.Client())
retrieved_documents_df = get_retrieved_documents(px.Client())

In [33]:
queries_df

,input,output,reference
context.span_id,,,
14fcacd026e9f882,What is the impact of ‘write_consistency_factor’ ?,"The `write_consistency_factor` impacts the reliability and consistency of write operations in a distributed system. When this factor is increased, it requires more replicas to acknowledge a write operation before it is considered successful and a response is sent to the client. This makes write operations more tolerant to network partitions, as it ensures that a larger number of replicas have the write data before proceeding. However, it also means that a higher number of active replicas are needed to perform write operations, which could potentially lead to higher latency or even failures in write operations if the required number of replicas is not available.","- `write_consistency_factor` - defines the number of replicas that must acknowledge a write operation before responding to the client. Increasing this value will make write operations tolerant to network partitions in the cluster, but will require a higher number of replicas to be active to perform write operations.\n\n### Write consistency factor\n\n\n\nThe `write_consistency_factor` represents the number of replicas that must acknowledge a write operation before responding to the client. It is set to one by default.\n\nIt can be configured at the collection's creation time.\n\n\n\n```http\n\nPUT /collections/{collection_name}\n\n{\n\n ""vectors"": {\n\n ""size"": 300,\n\n ""distance"": ""Cosine""\n\n },\n\n ""shard_number"": 6,\n\n ""replication_factor"": 2,\n\n ""write_consistency_factor"": 2,\n\n}\n\n```\n\n\n\n```python"
1a847e1aa16e4e15,What is significance of ‘on_disk_payload’ setting?,"The `on_disk_payload` setting is significant because it determines where payload data is stored. When set to `true`, it ensures that payload data is stored only on disk, rather than in RAM. This can be particularly useful for managing memory usage effectively when dealing with large payloads, as it helps to limit the amount of RAM required by not loading large payload data into memory.","* `on_disk_payload` - defines where to store payload data. If `true` - payload will be stored on disk only. Might be useful for limiting the RAM usage in case of large payload.\n\n* `quantization_config` - see [quantization](../../guides/quantization/#setting-up-quantization-in-qdrant) for details.\n\n\n\nDefault parameters for the optional collection parameters are defined in [configuration file](https://github.com/qdrant/qdrant/blob/master/config/config.yaml).\n\nThe payload data is loaded into RAM at service startup while disk and [RocksDB](https://rocksdb.org/) are used for persistence only.\n\nThis type of storage works quite fast, but it may require a lot of space to keep all the data in RAM, especially if the payload has large values attached - abstracts of text or even images.\n\n\n\nIn the case of large payload values, it might be better to use OnDisk payload storage."
a449f21559141637,How do you use ‘ordering’ parameter?,"The `ordering` parameter can be used with update and delete operations to ensure that these operations are executed in the same order on all replicas. To use this parameter, you would include it in your request to Qdrant. For example, when making an HTTP PUT request to update points in a collection, you would append `?ordering=strong` to the URL, like this:\n\n```\nPUT /collections/{collection_name}/points?ordering=strong\n```\n\nIn the body of the request, you would include the details of the points you want to update, such as their IDs, payloads, and vectors.\n\nWhen using the `ordering` parameter, Qdrant will route the operation to the leader replica of the shard and will not respond to the client until the operation has been completed. This ensures that all replicas process the update or delete operations in the same order, which is particularly useful for avoiding data inconsistencies when there are concurrent updates to the same documents.","- Write `o

In [34]:
retrieved_documents_df

context.trace_id  \
context.span_id  document_position                                     
7cab8e400cabe8e7 0                  2920c32bcf94359d7b2906641d8f2a10   
                 1                  2920c32bcf94359d7b2906641d8f2a10   
36d7d3481d63eb2f 0                  7817c0d936f861fa95d57d5965f6f831   
                 1                  7817c0d936f861fa95d57d5965f6f831   
4f34361dbde57463 0                  daa61b1e45237715bbd8eafcd09d6c2c   
                 1                  daa61b1e45237715bbd8eafcd09d6c2c   
bfd965ae080f81ce 0                  2e0440e73984c19e901106346153c857   
                 1                  2e0440e73984c19e901106346153c857   
e67bf6d53c9b6a51 0                  c96de30824656d78982669c4dcfc97ec   
                 1                  c96de30824656d78982669c4dcfc97ec   
8f7a4e52a6e6d701 0                  8ef88b9b258b5b5f0701af4d013d9148   
                 1                  8ef88b9b258b5b5f0701af4d013d9148   
0991009617cb49c3 0                  8432e0f2be7a3362c793486df97e83a8   
                 1                  8432e0f2be7a3362c793486df97e83a8   
5fd88bb374ffe79d 0                  b925efbd5bba561c6311573daca658cb   
                 1                  b925efbd5bba561c6311573daca658cb   
628b03885d84d954 0                  c88fa99870933175ce0729f6de90ae87   
                 1                  c88fa99870933175ce0729f6de90ae87   
ae05246a212b081b 0                  dd3dd56b00b59dd886c77503ac9e866c   
                 1                  dd3dd56b00b59dd886c77503ac9e866c   
a6aa9e13c32a83c7 0                  1c0ed28883f197e130197e0d9dfce1c0   
                 1                  1c0ed28883f197e130197e0d9dfce1c0   

                                                                                          input  \
context.span_id  document_position                                                                
7cab8e400cabe8e7 0                           What is the impact of ‘write_consistency_factor’ ?   
                 1                           What is the impact of ‘write_consistency_factor’ ?   
36d7d3481d63eb2f 0                           What is significance of ‘on_disk_payload’ setting?   
                 1                           What is significance of ‘on_disk_payload’ setting?   
4f34361dbde57463 0                                         How do you use ‘ordering’ parameter?   
                 1                                         How do you use ‘ordering’ parameter?   
bfd965ae080f81ce 0                                What is the purpose of ef_construct in HNSW ?   
                 1                                What is the purpose of ef_construct in HNSW ?   
e67bf6d53c9b6a51 0                            What is the purpose of ‘CreatePayloadIndexAsync’?   
                 1                            What is the purpose of ‘CreatePayloadIndexAsync’?   
8f7a4e52a6e6d701 0                                                 How does oversampling helps?   
                 1                                                 How does oversampling helps?   
0991009617cb49c3 0                                               What is ‘best_score’ strategy?   
                 1                                               What is ‘best_score’ strategy?   
5fd88bb374ffe79d 0                  What is difference between scalar and product quantization?   
                 1                  What is difference between scalar and product quantization?   
628b03885d84d954 0                                        Tell me about ‘always_ram’ parameter?   
                 1                                        Tell me about ‘always_ram’ parameter?   
ae05246a212b081b 0                                                   What is vaccum optimizer ?   
                 1                                                   What is vaccum optimizer ?   
a6aa9e13c32a83c7 0                                                   What is a Merge Optimizer?   
                 1                            

### **12. Define your evaluation model and your evaluators**

Next, define your evaluation model and your evaluators.

Evaluators are built on top of language models and prompt the LLM to assess the quality of responses, the relevance of retrieved documents, etc., and provide a quality signal even in the absence of human-labeled data. Pick an evaluator type and instantiate it with the language model you want to use to perform evaluations using our battle-tested evaluation templates.

In [35]:
eval_model = OpenAIModel(
    model="gpt-4-turbo-preview",
)
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_correctness_evaluator = QAEvaluator(eval_model)
relevance_evaluator = RelevanceEvaluator(eval_model)

hallucination_eval_df, qa_correctness_eval_df = run_evals(
    dataframe=queries_df,
    evaluators=[hallucination_evaluator, qa_correctness_evaluator],
    provide_explanation=True,
)
relevance_eval_df = run_evals(
    dataframe=retrieved_documents_df,
    evaluators=[relevance_evaluator],
    provide_explanation=True,
)[0]

px.Client().log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval_df),
    SpanEvaluations(eval_name="QA Correctness", dataframe=qa_correctness_eval_df),
)
px.Client().log_evaluations(DocumentEvaluations(eval_name="Relevance", dataframe=relevance_eval_df))

run_evals |          | 0/22 (0.0%) | ⏳ 00:00<? | ?it/s

run_evals |          | 0/22 (0.0%) | ⏳ 00:00<? | ?it/s

Your evaluations should now appear as annotations on the appropriate spans in Phoenix.

![A view of the Phoenix UI with evaluation annotations](https://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/evals/traces_with_evaluation_annotations.png)

### **13. Let's try Hybrid search now**

In [36]:
## Define a new collection to store your hybrid emebeddings
COLLECTION_NAME_HYBRID = "qdrant_docs_arize_hybrid"

In [26]:
##Reprocess documents with different settings if needed 
#documents = process_document_chunks(dataset , CHUNK_SIZE , CHUNK_OVERLAP)

In [27]:
#len(documents)

In [37]:
##List of supported sparse vector models
from fastembed.sparse.sparse_text_embedding import SparseTextEmbedding
SparseTextEmbedding.list_supported_models()

[{'model': 'prithvida/Splade_PP_en_v1',
  'vocab_size': 30522,
  'description': 'Misspelled version of the model. Retained for backward compatibility. Independent Implementation of SPLADE++ Model for English',
  'size_in_GB': 0.532,
  'sources': {'hf': 'Qdrant/SPLADE_PP_en_v1'}},
 {'model': 'prithivida/Splade_PP_en_v1',
  'vocab_size': 30522,
  'description': 'Independent Implementation of SPLADE++ Model for English',
  'size_in_GB': 0.532,
  'sources': {'hf': 'Qdrant/SPLADE_PP_en_v1'}}]

### **14. Ingest Sparse and Dense vectors into Qdrant**

Ingest sparse and dense vectors into Qdrant Collection.
We are using Splade++ model for Sparse Vector Model and default Fastembed model - bge-small-en-1.5 for dense embeddings. 

In [38]:
## Initializing the space to work with llama-index and related settings
import llama_index
from llama_index.core import Settings
from llama_index.vector_stores.qdrant import QdrantVectorStore
from fastembed.sparse.sparse_text_embedding import SparseTextEmbedding, SparseEmbedding
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from typing import List, Tuple

sparse_model_name = "prithivida/Splade_PP_en_v1"

# This triggers the model download
sparse_model = SparseTextEmbedding(model_name=sparse_model_name, batch_size=32)

## Computing sparse vectors
def compute_sparse_vectors(
    texts: List[str],
    ) -> Tuple[List[List[int]], List[List[float]]]:
    indices, values = [], []
    for embedding in sparse_model.embed(texts):
        indices.append(embedding.indices.tolist())
        values.append(embedding.values.tolist())
    return indices, values

## Creating a vector store with Hybrid search enabled
hybrid_vector_store = QdrantVectorStore(
    client=client,
    collection_name=COLLECTION_NAME_HYBRID,
    enable_hybrid=True,
    sparse_doc_fn=compute_sparse_vectors,
    sparse_query_fn=compute_sparse_vectors)

storage_context = StorageContext.from_defaults(vector_store=hybrid_vector_store)

Settings.embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
## Note : Ingesting sparse and dense vectors into Qdrant collection
## This block is to be used when you're creating a new collection if using an existing collection,
## Skip this block and execute the next one instead.
from phoenix.trace import suppress_tracing
with suppress_tracing():
    hybrid_vector_index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        show_progress=True
    )

In [39]:
## Note: Execute this block when using an existing collection
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from llama_index.core.indices.vector_store import VectorIndexRetriever

hybrid_vector_index = VectorStoreIndex.from_vector_store(vector_store=hybrid_vector_store)

In [40]:
## collection level operations
client.get_collection(COLLECTION_NAME_HYBRID)
#client.delete_collection(COLLECTION_NAME_HYBRID)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=8862, indexed_vectors_count=4429, points_count=4431, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors={'text-dense': VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None)}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors={'text-sparse': SparseVectorParams(index=SparseIndexParams(full_scan_threshold=None, on_disk=None))}), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_thread

In [41]:
## Check the number of documents matches the expected number of document chunks 
client.count(collection_name=COLLECTION_NAME_HYBRID)

CountResult(count=4431)

### **15. Hybrid Search with Qdrant**

In [42]:
## Before trying Hybrid search , lets try Sparse Vector Search Retriever 
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from llama_index.core.indices.vector_store import VectorIndexRetriever

sparse_retriever = VectorIndexRetriever(
    index=hybrid_vector_index,
    vector_store_query_mode=VectorStoreQueryMode.SPARSE,
    sparse_top_k=2,
)

## Pure sparse vector search
nodes = sparse_retriever.retrieve("What is a Merge Optimizer?")
for i, node in enumerate(nodes):
    print(i + 1, node.text, end="\n")

1 Such segments, for example, are created as copy-on-write segments during optimization itself.



It is also essential to have at least one small segment that Qdrant will use to store frequently updated data.

On the other hand, too many small segments lead to suboptimal search performance.



There is the Merge Optimizer, which combines the smallest segments into one large segment. It is used if too many segments are created.
2 The criteria for starting the optimizer are defined in the configuration file.



Here is an example of parameter values:



```yaml

storage:

  optimizers:

    # If the number of segments exceeds this value, the optimizer will merge the smallest segments.

    max_segment_number: 5

```



## Indexing Optimizer



Qdrant allows you to choose the type of indexes and data storage methods used depending on the number of records.


In [43]:
## Let's try Hybrid Search Retriever now using the 'alpha' parameter that controls the weightage between
## the dense and sparse vector search scores.
# NOTE: For hybrid search (0 for sparse search, 1 for dense search)

hybrid_retriever = VectorIndexRetriever(
    index=hybrid_vector_index,
    vector_store_query_mode=VectorStoreQueryMode.HYBRID,
    sparse_top_k=1,
    similarity_top_k=2,
    alpha=0.1,
)

In [44]:
## Let's try hybrid retriever 
nodes = hybrid_retriever.retrieve("What is a Merge Optimizer?")
for i, node in enumerate(nodes):
    print(i + 1, node.text, end="\n")

1 Such segments, for example, are created as copy-on-write segments during optimization itself.



It is also essential to have at least one small segment that Qdrant will use to store frequently updated data.

On the other hand, too many small segments lead to suboptimal search performance.



There is the Merge Optimizer, which combines the smallest segments into one large segment. It is used if too many segments are created.
2 ---

title: Optimizer

weight: 70

aliases:

  - ../optimizer

---



# Optimizer



It is much more efficient to apply changes in batches than perform each change individually, as many other databases do. Qdrant here is no exception. Since Qdrant operates with data structures that are not always easy to change, it is sometimes necessary to rebuild those structures completely.



Storage optimization in Qdrant occurs at the segment level (see [storage](../storage)).


In [451]:
# We shouldn't be modifying the alpha parameter after the retriever has been created
# but that's the easiest way to show the effect of the parameter
#hybrid_retriever._alpha = 0.1
#hybrid_retriever._alpha = 0.9

#nodes = hybrid_retriever.retrieve("What is merge optimizer?")
#for i, node in enumerate(nodes):
#    print(i + 1, node.text, end="\n")

In [45]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

#define response synthesizer
response_synthesizer = get_response_synthesizer()

#assemble query engine for hybrid retriever
hybrid_query_engine = RetrieverQueryEngine(
                        retriever=hybrid_retriever,
                        response_synthesizer=response_synthesizer,)

### **16. Re-Run Your Query Engine and View Your Traces in Phoenix**

Let's rerun the list of the baseline questions about Qdrant on the Hybrid Retriever. 

In [46]:
## Switching phoenix project space
from phoenix.trace import using_project

# Switch project to run evals
with using_project(HYBRID_RAG_PROJECT):
# All spans created within this context will be associated with the `HYBRID_RAG_PROJECT` project.

    ##Reuse the previously loaded dataset `qdrant_qa_question`
    
    for query in tqdm(qdrant_qa_question['question'][:10]):
        try:
          hybrid_query_engine.query(query)
        except Exception as e:
          pass



  0%|                                                                                                                                        | 0/10 [00:00<?, ?it/s]

 10%|████████████▊                                                                                                                   | 1/10 [00:05<00:47,  5.26s/it]

 20%|█████████████████████████▌                                                                                                      | 2/10 [00:12<00:52,  6.58s/it]

 30%|██████████████████████████████████████▍                                                                                         | 3/10 [00:21<00:51,  7.36s/it]

 40%|███████████████████████████████████████████████████▏                                                                            | 4/10 [00:29<00:47,  7.95s/it]

 50%|████████████████████████████████████████████████████████████████                                                                | 5/10 [00:35<00:35,  7.14s/it]

 6

In [48]:
print(f"🚀 Open the Phoenix UI if you haven't already: {session.url}")

🚀 Open the Phoenix UI if you haven't already: http://localhost:6006/


In [47]:
## Switching phoenix project space
from phoenix.trace import using_project


queries_df_hybrid = get_qa_with_reference(px.Client(), project_name=HYBRID_RAG_PROJECT)
retrieved_documents_df_hybrid = get_retrieved_documents(px.Client(), project_name=HYBRID_RAG_PROJECT)

In [48]:
queries_df_hybrid

,input,output,reference
context.span_id,,,
5c23a162c061df49,What is the impact of ‘write_consistency_factor’ ?,"The impact of the `write_consistency_factor` is that it defines the number of replicas that must acknowledge a write operation before the system responds to the client. If this value is increased, write operations will be more tolerant to network partitions within the cluster. However, this also means that a higher number of replicas need to be active for write operations to be performed successfully.","### Write consistency factor\n\n\n\nThe `write_consistency_factor` represents the number of replicas that must acknowledge a write operation before responding to the client. It is set to one by default.\n\nIt can be configured at the collection's creation time.\n\n\n\n```http\n\nPUT /collections/{collection_name}\n\n{\n\n ""vectors"": {\n\n ""size"": 300,\n\n ""distance"": ""Cosine""\n\n },\n\n ""shard_number"": 6,\n\n ""replication_factor"": 2,\n\n ""write_consistency_factor"": 2,\n\n}\n\n```\n\n\n\n```python\n\n- `write_consistency_factor` - defines the number of replicas that must acknowledge a write operation before responding to the client. Increasing this value will make write operations tolerant to network partitions in the cluster, but will require a higher number of replicas to be active to perform write operations."
37c98aebc845e8be,What is significance of ‘on_disk_payload’ setting?,"The `on_disk_payload` setting is significant because it determines where payload data is stored. When set to `true`, it ensures that payload data is stored only on disk, rather than in RAM. This can be particularly useful for managing memory usage effectively when dealing with large payloads, as it helps to limit the amount of RAM required by not loading large payload values into memory.","* `on_disk_payload` - defines where to store payload data. If `true` - payload will be stored on disk only. Might be useful for limiting the RAM usage in case of large payload.\n\n* `quantization_config` - see [quantization](../../guides/quantization/#setting-up-quantization-in-qdrant) for details.\n\n\n\nDefault parameters for the optional collection parameters are defined in [configuration file](https://github.com/qdrant/qdrant/blob/master/config/config.yaml).\n\nThe payload data is loaded into RAM at service startup while disk and [RocksDB](https://rocksdb.org/) are used for persistence only.\n\nThis type of storage works quite fast, but it may require a lot of space to keep all the data in RAM, especially if the payload has large values attached - abstracts of text or even images.\n\n\n\nIn the case of large payload values, it might be better to use OnDisk payload storage."
f805dcdab7f8a853,How do you use ‘ordering’ parameter?,"The `ordering` parameter can be used with update and delete operations to ensure that these operations are executed in the same order on all replicas. When this option is utilized, the operation is routed to the leader replica of the shard, and the system waits for a response from the leader before responding to the client. This is particularly useful for avoiding data inconsistencies that might arise from concurrent updates or deletions of the same documents.","Since the `filter` parameter is specified, the search is performed only among those points that satisfy the filter condition.\n\nSee details of possible filters and their work in the [filtering](../filtering) section.\n\n\n\nExample result of this API would be\n\n\n\n```json\n\n{\n\n ""result"": [\n\n { ""id"": 10, ""score"": 0.81 },\n\n { ""id"": 14, ""score"": 0.75 },\n\n { ""id"": 11, ""score"": 0.73 }\n\n ],\n\n ""status"": ""ok"",\n\n ""time"": 0.001\n\n}\n\n```\n\n\n\nThe `result` contains ordered by `score` list of found point ids.\n\n- Write `ordering` param, can be used with update and delete operations to ensure that the operations are executed in the same order on all replicas. If this option is used, Qdrant will route the operation to the leader 

In [49]:
retrieved_documents_df_hybrid

context.trace_id  \
context.span_id  document_position                                     
483939ac09cf8106 0                  05c01218d18e182252b4fefa2e82684d   
                 1                  05c01218d18e182252b4fefa2e82684d   
0a46933e8bfdf38d 0                  663f1d1540de82a7ad575e205b5ff803   
                 1                  663f1d1540de82a7ad575e205b5ff803   
bc1217192fe1f4d8 0                  5bea862dbb13dea05e5797a69d530e35   
                 1                  5bea862dbb13dea05e5797a69d530e35   
6f3cfcf870268b23 0                  851c0526ccef268680c9818a4dfebea3   
                 1                  851c0526ccef268680c9818a4dfebea3   
40a17423fd3bb548 0                  4d1a0bfa050b72d8ea801b5182d0ba1e   
                 1                  4d1a0bfa050b72d8ea801b5182d0ba1e   
84aa942596f6421d 0                  81f3b7adae989865c40765a1cfa27a0b   
                 1                  81f3b7adae989865c40765a1cfa27a0b   
5f4fa5b8af90db9d 0                  6227fd1001c0461e3b5d07b797f01f04   
                 1                  6227fd1001c0461e3b5d07b797f01f04   
1d478027476745f8 0                  ee0e5c425f5e0ebb1663351af27ed8c8   
                 1                  ee0e5c425f5e0ebb1663351af27ed8c8   
e5c9458cc34a3ed9 0                  5c10258cdd80bd339fe26149f978a774   
                 1                  5c10258cdd80bd339fe26149f978a774   
fc2a1e3793fa4f5e 0                  5cfbe060851ce9e9b6f916883017ce76   
                 1                  5cfbe060851ce9e9b6f916883017ce76   

                                                                                          input  \
context.span_id  document_position                                                                
483939ac09cf8106 0                           What is the impact of ‘write_consistency_factor’ ?   
                 1                           What is the impact of ‘write_consistency_factor’ ?   
0a46933e8bfdf38d 0                           What is significance of ‘on_disk_payload’ setting?   
                 1                           What is significance of ‘on_disk_payload’ setting?   
bc1217192fe1f4d8 0                                         How do you use ‘ordering’ parameter?   
                 1                                         How do you use ‘ordering’ parameter?   
6f3cfcf870268b23 0                                What is the purpose of ef_construct in HNSW ?   
                 1                                What is the purpose of ef_construct in HNSW ?   
40a17423fd3bb548 0                            What is the purpose of ‘CreatePayloadIndexAsync’?   
                 1                            What is the purpose of ‘CreatePayloadIndexAsync’?   
84aa942596f6421d 0                                                 How does oversampling helps?   
                 1                                                 How does oversampling helps?   
5f4fa5b8af90db9d 0                                               What is ‘best_score’ strategy?   
                 1                                               What is ‘best_score’ strategy?   
1d478027476745f8 0                  What is difference between scalar and product quantization?   
                 1                  What is difference between scalar and product quantization?   
e5c9458cc34a3ed9 0                                        Tell me about ‘always_ram’ parameter?   
                 1                                        Tell me about ‘always_ram’ parameter?   
fc2a1e3793fa4f5e 0                                                   What is vaccum optimizer ?   
                 1                                                   What is vaccum optimizer ?   

                                                                                                                                                                                                                                                                                                

### **17. Define your evaluation model and your evaluators for Hybrid Search**

Next, define your evaluation model and your evaluators.

Evaluators are built on top of language models and prompt the LLM to assess the quality of responses, the relevance of retrieved documents, etc., and provide a quality signal even in the absence of human-labeled data. Pick an evaluator type and instantiate it with the language model you want to use to perform evaluations using our battle-tested evaluation templates.

In [50]:


# all spans created within this context will be associated with the `HYBRID_RAG_PROJECT` project.
eval_model = OpenAIModel(
    model="gpt-4-turbo-preview",
)
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_correctness_evaluator = QAEvaluator(eval_model)
relevance_evaluator = RelevanceEvaluator(eval_model)

hallucination_eval_df_hybrid, qa_correctness_eval_df_hybrid = run_evals(
    dataframe=queries_df_hybrid,
    evaluators=[hallucination_evaluator, qa_correctness_evaluator],
    provide_explanation=True,
)
relevance_eval_df_hybrid = run_evals(
    dataframe=retrieved_documents_df_hybrid,
    evaluators=[relevance_evaluator],
    provide_explanation=True,
)[0]

px.Client().log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval_df_hybrid),
    SpanEvaluations(eval_name="QA Correctness", dataframe=qa_correctness_eval_df_hybrid),
    project_name=HYBRID_RAG_PROJECT,
)
px.Client().log_evaluations(DocumentEvaluations(eval_name="Relevance", dataframe=relevance_eval_df_hybrid),
                            project_name=HYBRID_RAG_PROJECT)

run_evals |          | 0/20 (0.0%) | ⏳ 00:00<? | ?it/s

run_evals |          | 0/20 (0.0%) | ⏳ 00:00<? | ?it/s